In [72]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"  # You can use other BERT variants like 'bert-large-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # For binary classification

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import data_file

15
(15, 384)
15
Vectorstore made


In [3]:
data_list=data_file.data1
data_list

[{'content': 'AI-Driven Personalization in Online Education\nPlatforms: Harnessing the Power of Artificial\nIntelligence to Revolutionize Learning Experiences\nAbstract\nAI-driven personalization is revolutionizing online education platforms by offer-\ning tailored learning experiences to individual students. This approach leverages\nmachine learning algorithms to analyze student behavior, learning patterns, and\nknowledge gaps, thereby creating a unique learning pathway for each student. How-\never, our research takes an unconventional turn by incorporating an AI-generated\ndreamscape into the personalization framework, where students’ subconscious\nthoughts and desires are used to create a more immersive learning environment.\nWe propose that this unorthodox method can lead to increased student engagement\nand improved learning outcomes, despite its apparent lack of logical connection to\ntraditional educational paradigms.\n1 Introduction\nThe advent of online education platforms has

In [4]:
data_X=[]
data_Y=[]
for element in data_list:
    data_X.append(element['content'])
    data_Y.append(element['label'])
data_X

['AI-Driven Personalization in Online Education\nPlatforms: Harnessing the Power of Artificial\nIntelligence to Revolutionize Learning Experiences\nAbstract\nAI-driven personalization is revolutionizing online education platforms by offer-\ning tailored learning experiences to individual students. This approach leverages\nmachine learning algorithms to analyze student behavior, learning patterns, and\nknowledge gaps, thereby creating a unique learning pathway for each student. How-\never, our research takes an unconventional turn by incorporating an AI-generated\ndreamscape into the personalization framework, where students’ subconscious\nthoughts and desires are used to create a more immersive learning environment.\nWe propose that this unorthodox method can lead to increased student engagement\nand improved learning outcomes, despite its apparent lack of logical connection to\ntraditional educational paradigms.\n1 Introduction\nThe advent of online education platforms has revolutioni

In [65]:
def sliding_window_tokenize(text, max_length=512, overlap=256):
    tokens = tokenizer.encode(text)
    windows = []
    for i in range(0, len(tokens), max_length - overlap):
        window = tokens[i:i + max_length]
        windows.append(window)
    return windows
tokenized_cont={}
for i in range(len(data_X)):
    windows = sliding_window_tokenize(data_X[i], max_length=512, overlap=256)
    tokenized_cont[i]=windows

In [66]:
sum=0
for i in range(len(tokenized_cont)):
    sum+=len(tokenized_cont[i])
sum

456

In [67]:
from torch.utils.data import Dataset
import torch

# Define the Dataset class
class BinaryClassificationDataset(Dataset):
    def __init__(self, tokenized_windows, labels):
        """
        Args:
        - tokenized_windows: Pre-tokenized windows of text (List[List[int]]).
        - labels: Corresponding labels for each window (List[int]).
        """
        self.tokenized_windows = tokenized_windows
        self.labels = labels

    def __len__(self):
        return len(self.tokenized_windows)

    def __getitem__(self, idx):
        """
        Retrieve a single data sample.
        Args:
        - idx: Index of the data sample.
        Returns:
        - A dictionary with 'input_ids', 'attention_mask', and 'labels'.
        """
        input_ids = self.tokenized_windows[idx]
        label = self.labels[idx]

        # Generate attention mask (1 for real tokens, 0 for padding)
        attention_mask = [1] * len(input_ids) + [0] * (512 - len(input_ids))

        # Pad input_ids to max_length (512)
        padded_input_ids = input_ids + [0] * (512 - len(input_ids))

        return {
            'input_ids': torch.tensor(padded_input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Prepare tokenized windows and labels
all_windows = []
all_labels = []

# Dynamically populate windows and replicate labels
for idx, windows in tokenized_cont.items():
    all_windows.extend(windows)  # Add all windows for the sample
    all_labels.extend([data_Y[idx]] * len(windows))  # Replicate the label for each window

# Verify size consistency
assert len(all_windows) == len(all_labels), "Mismatch between windows and labels!"

# Initialize the dataset
train_dataset = BinaryClassificationDataset(all_windows, all_labels)

# Check dataset size and a sample
print(f"Dataset size: {len(train_dataset)}")
print(f"Sample data: {train_dataset[0]}")




Dataset size: 456
Sample data: {'input_ids': tensor([  101,  9932,  1011,  5533,  3167,  3989,  1999,  3784,  2495,  7248,
         1024, 17445,  2075,  1996,  2373,  1997,  7976,  4454,  2000,  4329,
         4697,  4083,  6322, 10061,  9932,  1011,  5533,  3167,  3989,  2003,
         4329,  6026,  3784,  2495,  7248,  2011,  3749,  1011, 13749, 21727,
         4083,  6322,  2000,  3265,  2493,  1012,  2023,  3921, 21155,  2015,
         3698,  4083, 13792,  2000, 17908,  3076,  5248,  1010,  4083,  7060,
         1010,  1998,  3716, 16680,  1010,  8558,  4526,  1037,  4310,  4083,
        12732,  2005,  2169,  3076,  1012,  2129,  1011,  2412,  1010,  2256,
         2470,  3138,  2019, 23693,  2735,  2011, 13543,  2019,  9932,  1011,
         7013,  5544, 19464,  2046,  1996,  3167,  3989,  7705,  1010,  2073,
         2493,  1521, 27952,  4301,  1998, 14714,  2024,  2109,  2000,  3443,
         1037,  2062, 10047, 16862,  3512,  4083,  4044,  1012,  2057, 16599,
         2008,  202

In [68]:
type(train_dataset)

__main__.BinaryClassificationDataset

In [69]:
len(train_dataset)

456

In [70]:
from transformers import Trainer, TrainingArguments
import accelerate
import torch
accelerate.__version__

'1.0.1'

In [73]:
training_args = TrainingArguments(
    output_dir='./results',            # output directory for model predictions and checkpoints
    num_train_epochs=1,                # number of training epochs
    per_device_train_batch_size=1,     # batch size for training
    per_device_eval_batch_size=1,      # batch size for evaluation
    warmup_steps=500,                  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                 # strength of weight decay
    logging_dir='./logs',              # directory for storing logs
    logging_steps=1,
)

# Setup Trainer
trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset,         # Pass the dataset
)

# Fine-tune the model
trainer.train()

  0%|          | 0/456 [00:00<?, ?it/s]

{'loss': 0.8526, 'grad_norm': 25.97369956970215, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.0}
{'loss': 0.9037, 'grad_norm': 16.417007446289062, 'learning_rate': 2.0000000000000002e-07, 'epoch': 0.0}
{'loss': 0.4703, 'grad_norm': 11.495830535888672, 'learning_rate': 3.0000000000000004e-07, 'epoch': 0.01}
{'loss': 0.437, 'grad_norm': 19.418743133544922, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.01}
{'loss': 0.9303, 'grad_norm': 16.288583755493164, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.01}
{'loss': 0.9317, 'grad_norm': 23.142452239990234, 'learning_rate': 6.000000000000001e-07, 'epoch': 0.01}
{'loss': 0.5181, 'grad_norm': 8.994649887084961, 'learning_rate': 7.000000000000001e-07, 'epoch': 0.02}
{'loss': 0.6319, 'grad_norm': 33.24098205566406, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.02}
{'loss': 0.3894, 'grad_norm': 9.09587574005127, 'learning_rate': 9e-07, 'epoch': 0.02}
{'loss': 0.9309, 'grad_norm': 19.48393440246582, 'learning_rate': 1.0000000

TrainOutput(global_step=456, training_loss=0.16829055353519562, metrics={'train_runtime': 642.8058, 'train_samples_per_second': 0.709, 'train_steps_per_second': 0.709, 'total_flos': 119978641244160.0, 'train_loss': 0.16829055353519562, 'epoch': 1.0})

In [74]:
# Assuming you have a validation dataset, pass it here
trainer.evaluate(train_dataset)  # Pass your validation dataset (similar to train_dataset)


  0%|          | 0/456 [00:00<?, ?it/s]

{'eval_loss': 0.0007896256865933537,
 'eval_runtime': 127.6488,
 'eval_samples_per_second': 3.572,
 'eval_steps_per_second': 3.572,
 'epoch': 1.0}

In [75]:
# Save the model and tokenizer
model.save_pretrained('./binary_classification_model')
tokenizer.save_pretrained('./binary_classification_model')

('./binary_classification_model/tokenizer_config.json',
 './binary_classification_model/special_tokens_map.json',
 './binary_classification_model/vocab.txt',
 './binary_classification_model/added_tokens.json')

In [76]:
import torch

def prepare_windows_for_model(windows, tokenizer, max_length=512):
    """
    Converts tokenized windows into model-compatible tensors.
    
    Args:
    - windows: List of tokenized windows.
    - tokenizer: Pretrained tokenizer object.
    - max_length: int, maximum sequence length for the model.
    
    Returns:
    - input_tensors: List of dictionaries with 'input_ids' and 'attention_mask'.
    """
    input_tensors = []
    for window in windows:
        # Decode tokens back into text for the tokenizer
        text_window = tokenizer.decode(window, skip_special_tokens=True)
        
        # Tokenize and encode for the model
        encoding = tokenizer(
            text_window,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors="pt",
        )
        input_tensors.append({
            'input_ids': encoding['input_ids'].squeeze(0),  # Remove extra batch dimension
            'attention_mask': encoding['attention_mask'].squeeze(0)
        })
    return input_tensors


In [77]:
from pypdf import PdfReader
import os

In [111]:
input_text=""
with open('./Papers/P047.pdf', "rb") as f:
    reader=PdfReader(f)
    for pages in reader.pages:
        input_text+=pages.extract_text()
input_text    

'Optimizing System Design Principles on Inverted\nHarmonica Tuning Frequencies\nAbstract\nThe intricacies of system design intersect with the existential implications of\nquantum cheese, which in turn, influences the aerodynamic properties of flamingos,\nand conversely, the abstract notion of colorless green ideas sleeping furiously, while\nthe ontological status of furniture arrangements in Scandinavian apartments remains\nan enigma, alongside the theoretical frameworks governing the migration patterns\nof narwhals and the surreptitious culinary habits of extraterrestrial beings, all of\nwhich converge to form a holistic understanding of the synergistic relationships\nbetween disparate entities, transcending the boundaries of reality and fantasy, in a\nrealm where the cartography of lost socks and the topological analysis of coffee\ncreamer dispensers serve as metaphors for the human condition, and ultimately, the\nsearch for meaning in a seemingly meaningless world, through the decon

In [112]:
# Load the fine-tuned model
model = BertForSequenceClassification.from_pretrained('./binary_classification_model')
tokenizer = BertTokenizer.from_pretrained('./binary_classification_model')
# Example inference
prompt=input_text+"\n\n Generate publishable or non publishable like the following: \n"+data_X[0]+"\n"+"0"
tokenized_text=prepare_windows_for_model(sliding_window_tokenize(prompt),tokenizer)
output = model(input_ids=tokenized_text[0]['input_ids'].unsqueeze(0),attention_mask=tokenized_text[0]['attention_mask'].unsqueeze(0))

# Convert logits to probabilities (if needed)
logits = output.logits
probabilities = torch.nn.Softmax(dim=1)(logits)
predicted_class = torch.argmax(probabilities, dim=1)

print(predicted_class)


Token indices sequence length is longer than the specified maximum sequence length for this model (16628 > 512). Running this sequence through the model will result in indexing errors


tensor([0])


1